# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * FastText
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import FastText, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

/Users/hyunyoun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## TaggedDocument For FastText

In [6]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Load Data

### Path

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentsPath = '/Volumes/disk1/data/daum_Comments/'
    naverCommentsPath = '/Volumes/disk1/data/naver_Comments/'
    outcomeDaumCommentsPath = '/Volumes/disk1/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = '/Volumes/disk1/outcome_comments_for_naver/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentsPath = 'd:/data/daum_Comments/'
    naverCommentsPath = 'd:/data/naver_Comments/'
    outcomeDaumCommentsPath = 'd:/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = 'd:/outcome_comments_for_naver/'

### News

In [8]:
os.listdir(news_senti_outcome)

['naver_news_sentiment_analysis.csv', 'daum_news_sentiment_analysis.csv']

In [9]:
# Naver
naverData = pd.read_csv(os.path.join(news_senti_outcome, 'naver_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
naverData['site'] = ['Naver'] * naverData.shape[0]
reNaverData = naverData[naverData.number_of_crawled_comment != 0]
print (reNaverData.shape)
reNaverData.head()

(15015, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1465,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","['외무성', '핵전쟁', '대변인']","{'미국', '조선반도', '핵전쟁', '북한', '중앙', '도화선', '고위',...",86.0,22.0,negative,Naver
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2062,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"['예산', '예결위', 'soc']","{'호남', '예산안', '의원', '정부안', '증액', '지역구', '국민의당'}",46.0,62.0,positive,Naver
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,536,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"['최경환', '구속영장', '국가정보원']","{'구속영장 청구', '조사', '의원', '정기국회', '국정원장', '검찰', ...",77.0,31.0,negative,Naver
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,133,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"['이슈 · 최재형 감사원장 내정', '감사원장', '최재형', '감사원']","{'공직 사회', '법관', '생활', '감사원장', '지명', '후보자'}",39.0,69.0,positive,Naver
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,932,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"['김정은', 'b-1b', '한반도']","{'공장', '양강도', '사무차장', '삼지연', '시찰', '훈련', '접경',...",72.0,36.0,negative,Naver


In [10]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
reDaumData = daumData[daumData.number_of_crawled_comment != 0]
print (reDaumData.shape)
reDaumData.head()

(9369, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,903,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","['부작용', '비아그라', '아일랜드']","{'건강', '부작용', '공장', '남성들', '지역', '세보 효과', '연기'}",50.0,58.0,positive,daum
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"['커피', '낮잠', '음식물']","{'건강', '커피', '자세', '치아', '철분', '디스크', '점심 식사',...",81.0,27.0,negative,daum
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,807,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"['살인혐의', '철원', '검찰송치']","{'앙심', '남편', '성관계', '주변', '진술', '지인', '범행', '아...",99.0,9.0,negative,daum
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,368,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","['신영자', '갑질', '롯데백화점']","{'네이처리퍼블릭', '롯데', '징역', '유통업체', '신영자 이사장', '검찰...",77.0,31.0,negative,daum
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,367,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"['공범', '살인', '과자']","{'폭행', '혐의', '과자', '범행', '쪽지', '남자친구', '경찰', '...",102.0,6.0,negative,daum


#### 댓글
* 예시)

In [11]:
df = pd.read_csv(os.path.join(naverCommentsPath, '5a29c445588c132954d1973a'+'.csv'), encoding='utf-8', index_col=None, header = 0)
df.head()

,_id,category,comments,date,rank,site,공감,비공감
0,5a29c449588c132954d19ba1,정치,6.25때와는 확실히 틀릴것이다.,2017.12.07,1,Naver,1,0
1,5a29c449588c132954d19daa,정치,60대 알바들 오늘도 수고가 많다. 근데 니들 이제 조사들어가겠더라? 업보다 업보야...,2017.12.07,1,Naver,1,4
2,5a29c448588c132954d1988a,정치,"625때 중국에 핵폭탄만 투하했다면 통일 됐을건데, 트루먼이 말려서 통일도 못하고 ...",2017.12.07,1,Naver,7,0
3,5a29c448588c132954d199b6,정치,625때에도 인천상륙작전실시하기 전에 서해안 여러곳을 가짜로 상륙하려고 시도를 했었...,2017.12.07,1,Naver,3,0
4,5a29c449588c132954d19ac7,정치,9.11테러때 납작 엎드려서 부지한 목숨이 아깝지 않은가보네 ㅋㅋㅋㅋㅋ,2017.12.07,1,Naver,2,0


## FastText Model

### Twitter & Mecab
> * 메모리, 시간상의 문제점
>> * 메모리는 dataframe apply로 하면 kernel이 죽어버림.
>> * 시간 for 문으로 할 수 있는건 한번에 해버리는게 낫겠지? 

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName_ct = './data/pre_data/train_test_Data/pre_data_train_for_fastText_sentiment_by_ct.pickled'
train_ct = pickle.load(open(trainName_ct, 'rb'))
tfidf_ct = bm.Build_tfidf(train_ct)
del train

100%|██████████| 442359/442359 [00:00<00:00, 1334417.99it/s]


(442359, 159010)
vocab size : 159010


#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName_mecab = './data/pre_data/train_test_Data/pre_data_train_for_fastText_sentiment_by_mecab.pickled'
train_mecab = pickle.load(open(trainName_mecab, 'rb'))
tfidf_mecab = bm.Build_tfidf(train_mecab)
del train

100%|██████████| 442359/442359 [00:00<00:00, 1482191.56it/s]


(442359, 162003)
vocab size : 162003


#### Load Model

In [12]:
taggerName_ct = 'ct'
print ( '{}'.format(taggerName_ct))
model1_ct = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model1_ct_Name = bm.Return_ModelName('fastText', model1_ct,'ct')
print ('Load Model')
model2_ct = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model2_ct_Name = bm.Return_ModelName('fastText', model2_ct,'ct')
print ('Load Model')
model3_ct = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')
model3_ct_Name = bm.Return_ModelName('fastText', model3_ct,'ct')
print ('Load Model')

taggerName_mecab = 'mecab'
print ( '{}'.format(taggerName_mecab))
model1_mecab = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-mecab.model')
model1_mecab_Name = bm.Return_ModelName('fastText', model1_mecab,'mecab')
print ('Load Model')
model2_mecab = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-mecab.model')
model2_mecab_Name = bm.Return_ModelName('fastText', model2_mecab,'mecab')
print ('Load Model')
model3_mecab = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-mecab.model')
model3_mecab_Name = bm.Return_ModelName('fastText', model3_mecab,'mecab')
print ('Load Model')

Load Model1
Load Model2
Load Model3


In [ ]:
for idx in reNaverData.index:
    commentFile = os.path.join(naverCommentsPath, idx+'.csv')
    if os.path.isfile(commentFile):
        

#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled', 'wb'))

#### model1

In [13]:
modelName = bm.Return_ModelName('fastText', model1,'ct')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_1
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:07<00:00, 20787.15it/s]


running time : 0:00:07.914689
Vectorizing Data


9372it [16:48,  9.29it/s]


scaling Data
total running time : 0:16:56.839356


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 623853.63it/s]


9372/9372 [==============================] - ETA: 40 - ETA: 0 - ETA:  - ETA:  - 0s 33us/step


9372it [00:00, 44971.77it/s]


9372/9372 [==============================] - ETA: 1: - ETA: 1s - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 57us/step


9372it [00:00, 176563.40it/s]


Wall time: 21.5 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:02<00:00, 70060.78it/s]


running time : 0:00:02.325316
Vectorizing Data


15120it [11:31, 21.85it/s]


scaling Data
total running time : 0:11:35.294469


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 350855.74it/s]


15120/15120 [==============================] - ETA: 2: - ETA: 3s - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 64us/step


15120it [00:00, 629008.32it/s]


15120/15120 [==============================] - ETA: 49 - ETA: 1 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 31us/step


15120it [00:00, 629058.23it/s]


Wall time: 30.9 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('fastText', model2,'ct')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:05<00:00, 27980.10it/s]


running time : 0:00:05.853044
Vectorizing Data


9372it [05:33, 28.13it/s]


scaling Data
total running time : 0:05:39.259452


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 623823.93it/s]


9372/9372 [==============================] - ETA: 32 - ETA: 0 - ETA:  - ETA:  - 0s 29us/step


9372it [00:00, 222804.87it/s]


9372/9372 [==============================] - ETA: 16 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 36us/step


9372it [00:00, 584877.28it/s]


Wall time: 20.6 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:01<00:00, 105873.29it/s]


running time : 0:00:01.539473
Vectorizing Data


15120it [03:05, 81.54it/s]


scaling Data
total running time : 0:03:07.277000


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 308099.05it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 19us/step


15120it [00:00, 603847.50it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 24us/step


15120it [00:00, 628790.03it/s]


Wall time: 32.6 s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('fastText', model3,'ct')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:02<00:00, 57702.31it/s]


running time : 0:00:02.822298
Vectorizing Data


9372it [04:00, 39.04it/s]


scaling Data
total running time : 0:04:03.101711


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 359929.84it/s]


9372/9372 [==============================] - ETA: 26 - ETA: 0 - ETA:  - ETA:  - 0s 27us/step


9372it [00:00, 623883.33it/s]


9372/9372 [==============================] - ETA: 25 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 33us/step


9372it [00:00, 623853.63it/s]


Wall time: 23.4 s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:01<00:00, 114794.48it/s]


running time : 0:00:01.421138
Vectorizing Data


15120it [03:25, 73.73it/s]


scaling Data
total running time : 0:03:26.819750


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 629076.95it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 18us/step


15120it [00:00, 628796.27it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 25us/step


15120it [00:00, 628790.03it/s]


Wall time: 37.3 s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

### Mecab

#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'wb'))

#### Load Model

#### model1

In [13]:
modelName = bm.Return_ModelName('fastText', model1,'mecab')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_1
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:05<00:00, 30233.65it/s]


running time : 0:00:05.661371
Vectorizing Data


9372it [17:47,  8.78it/s]


scaling Data
total running time : 0:17:53.867761


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 343331.18it/s]


9372/9372 [==============================] - ETA: 37 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 36us/step


9372it [00:00, 143966.40it/s]


9372/9372 [==============================] - ETA: 41 - ETA: 3 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 54us/step


9372it [00:00, 119787.23it/s]


Wall time: 24.2 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:03<00:00, 51421.00it/s]


running time : 0:00:03.228807
Vectorizing Data


15120it [07:09, 35.20it/s]


scaling Data
total running time : 0:07:13.156170


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 629070.71it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 17us/step


15120it [00:00, 603657.83it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 35us/step


15120it [00:00, 126679.43it/s]


Wall time: 27 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('fastText', model2,'mecab')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:03<00:00, 46049.51it/s]


running time : 0:00:03.605978
Vectorizing Data


9372it [06:48, 22.97it/s]


scaling Data
total running time : 0:06:51.856900


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 641078.61it/s]


9372/9372 [==============================] - ETA: 22 - ETA: 0 - ETA:  - ETA:  - 0s 26us/step


9372it [00:00, 623893.23it/s]


9372/9372 [==============================] - ETA: 15 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 29us/step


9372it [00:00, 668486.59it/s]


Wall time: 20.3 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:01<00:00, 107998.55it/s]


running time : 0:00:01.539425
Vectorizing Data


15120it [05:40, 44.40it/s]


scaling Data
total running time : 0:05:42.461675


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 656425.01it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 17us/step


15120it [00:00, 656418.21it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 26us/step


15120it [00:00, 629083.19it/s]


Wall time: 32.5 s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('fastText', model3,'mecab')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:04<00:00, 39179.53it/s]


running time : 0:00:04.236385
Vectorizing Data


9372it [04:50, 32.29it/s]


scaling Data
total running time : 0:04:54.642550


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 584885.98it/s]


9372/9372 [==============================] - ETA: 22 - ETA: 0 - ETA:  - ETA:  - 0s 25us/step


9372it [00:00, 668520.70it/s]


9372/9372 [==============================] - ETA: 23 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 33us/step


9372it [00:00, 623883.33it/s]


Wall time: 22.1 s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:01<00:00, 121210.36it/s]


running time : 0:00:01.372066
Vectorizing Data


15120it [04:15, 59.10it/s]


scaling Data
total running time : 0:04:17.519591


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 629045.75it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 17us/step


15120it [00:00, 603657.83it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 25us/step


15120it [00:00, 503232.61it/s]


Wall time: 35.5 s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict